<center><img src="../img/ICHEC_Logo.png" alt="Drawing" style="width: 500px;"/>

# <center>C Foreign Function Interface for Python - `cffi`</center>

***
## <center>Overview<center/>

Coding in higher level languages like Python has advantages and disadvantages. One of the latter is that many useful libaries are written in lower level languages like C. 
    
There are a few things one can do.
    
<details>
    <summary markdown="span">There are a few options...</summary>
<br>
   - Port all or part of the library to your language of choice<br>
   - Write an extension in C to bridge the gap between the library and your language<br>
   - Wrap the library using your language's <b>foreign function interface</b> (FFI) support<br>
<br>
<details>
    <summary markdown="span">So why are FFI wrappers best, usually?</summary>
<br>
   - Easier to write and maintain than a C-extension and more portable<br>
   - The FFI meaning refers to the ability for code written in one language, also known as the <b>host</b> (eg. Python) to access and invoke functions from another <b>guest</b> language (eg. C, Fortran)<br>
   - FFI utilities are common across multiple languages (Ruby, Lisp, Perl, Java)<br>
<br>
<details>
    <summary markdown="span">But there are times when not to use FFI</summary>
<br>
   - Implementation of your own low-level/optimised code, especially if you need to write custom code to directly process huge arrays, in order to get the performance you want<br>
   - Delicate callbacks from the guest language into the host. Some complex callbacks can be difficult<br>
   - Library makes use of compile-time or preprocessor features (eg. C macros)<br>
</details>
</details>
</details>

***
## <center>`cffi`

CFFI is an external package for Python that provides a C Foreign Function Interface for Python, and allows one to interact with almost any C code from Python (C++ not supported).

The user needs to add C-like declarations (not static) to the Python code. Some knowledge of C is highly recommended.

CFFI has two different **main** modes, ABI and API.

- **ABI (Application Binary Interface)**: Accesses the library at the binary level. Considered the easier option, but has issues with non-Windows platforms. Function calls need to go through `libffi` library which is slow.
- **API (Application Programmer Interface)**: A separate compilation step with a C compiler is utilised. Trickier option, but faster with C compiler. This mode compilers CPython C wrapper that directly involves target function

The goal is not about embedding executable C code in Python, but instead calls existing C libraries from Python.

There are 'sub-modes' of CFFI, each of ABI and API have **out-of-line** and **in-line** options. The modes that you should bear in mind are as follows:

- **ABI, in line**: Easiest
- **API, out of line**: Purely for performance

### Using the FFI Interface

The CFFI references can be found in the [documentation](https://cffi.readthedocs.io/en/latest/ref.html), however the most commonly used functions are as follows;

When 
- `ffi.cast("C-type", value)` - The value is casted between integers or pointers of any type
- `ffi.dlopen(libpath, [flags]` - opens and returns a handle to a dynamic library
- `ffibuilder`


### `cffi` Modes

* **ABI**
    * 'Application Binary Interface'
    * Accesses the libraries at the binary level
    * Easier but can lead to problems (speed)
    * **'in-line mode'** - everything set up every time you import your Python code


* **API**
    * 'Application Programming Interface'
    * Accesses the libraries with a C compiler (which they should be doing!)
    * Faster as a result of C compiler
    * **'out-of-line mode'** - separate step of preparation that produces a module which your main program imports


### Importing an existing library<center/>
    
Here we are going to import the `sqrt` function from the C standard math library

In [ ]:
from cffi import FFI
ffi = FFI()

lib = ffi.dlopen("libm.so.6")
ffi.cdef("""float sqrtf(float x);""")
a = lib.sqrtf(4)
print(a)

<div class="alert alert-block alert-info">
    Library files have the extension <b>.so</b> for Windows and UNIX systems and <b>.dylib</b> for Macs
<div/>

### Creating and using your own library

***
#### **ABI (in-line)** - "Easy" with poor performance

**Step 1: Create your C file**

In [ ]:
%%writefile ABI_add.c
#include <stdio.h>

void add(double *a, double *b, int n)
{
    int i;
    for (i=0; i<n; i++) {
        a[i] += b[i];
    }
}

**Step 2: Create a** ***Makefile*** **for your library**

In [ ]:
%%writefile Makefile
CC=gcc
CFLAGS=-fPIC -O3
LDFLAGS=-shared

mylib_add.so: ABI_add.c
	$(CC) -o ABI_add.so $(LDFLAGS) $(CFLAGS) ABI_add.c

**Step 3: Create your library by using the `make` command in the terminal**

- Use `ls` to see our directory beforehand

In [ ]:
!ls

- Run the `make` command

In [ ]:
!make

- Use `ls` to see the changes

In [ ]:
!ls

<br>

- Our library has been added as a `.so` file (or `.dylib` depending on operating system)

**Step 4: Import the library and cast the variables**

Here we need to use specific functions to get the C library we created working;

- `ffi.dlopen(libpath, [flags])` - opens and returns a handle to a dynamic library, which we can then use later to call the functions from that library
- `ffi.cdef("C-type", value)` - creates a new ffi object with the declaration of function
- `ffi.cast("C-type", value)` - The value is casted between integers or pointers of any type
- `ffi.from_buffer([cdecl], python_buffer)` - Return an array of cdata that points to the data of the given Python object

In [ ]:
from cffi import FFI
import numpy as np
import time 

ffi = FFI()
lib = ffi.dlopen('./ABI_add.so')
ffi.cdef("void add(double *, double *, int);")


t0=time.time()
a = np.arange(0,200000,0.1)
b = np.ones_like(a)

# "pointer" objects
aptr = ffi.cast("double *", ffi.from_buffer(a))
bptr = ffi.cast("double *", ffi.from_buffer(b))

lib.add(aptr, bptr, len(a))
print("a + b = ", a)
t1=time.time()

print ("\ntime taken for ABI in line", t1-t0)

***

#### **API (out of line)** - "Hard" with improved performance 

**Step 1: Create a python** ***build*** **file**

- Set up an `ffibuilder` handle
- `ffi.cdef("C-type", value)` - Like before, creates a new ffi object
- `ffibuilder.set_source`gives the name of the python extension module to produce, and some C source code as a string.  This C code needs to make the declarated functions, types and globals available

In [ ]:
%%writefile API_add_build.py

import cffi
ffibuilder = cffi.FFI()
ffibuilder.cdef("""void API_add(double *, double *, int);""")
ffibuilder.set_source("out_of_line._API_add", r"""
  void API_add(double *a, double *b, int n)
  {
      int i;
      for (i=0; i<n; i++){
          a[i] += b[i];
          }
          /* or some algorithm that is seriously faster in C than in Python */
  }
  """)


if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

**Step 2: Run the** ***build*** **file**

In [ ]:
!python API_add_build.py

<br>

- This creates a new directory, here called `out_of_line`, which is the name of our module

In [ ]:
!ls

<br>

- Checking inside `out_of_line`...

In [ ]:
!ls -l out_of_line/

<br>

- We have our library `.so`/`.dylib` file , generated `.c` and `.o` files

**Step 3: Import the library and cast the variables**

In [ ]:
import numpy as np 
import time
from out_of_line._API_add import ffi, lib

t0=time.time()
a = np.arange(0,200000,0.1)
b = np.ones_like(a)

# "pointer" objects
aptr = ffi.cast("double *", ffi.from_buffer(a))
bptr = ffi.cast("double *", ffi.from_buffer(b))

lib.API_add(aptr, bptr, len(a))
print("a + b = ", a)
t1=time.time()
print("\ntime taken for API out of line", t1-t0)

***

## Summary of different methods

### ABI - in line

* Create/Have a `.c` file containing your C code
* Have a `MAKEFILE` with compiler flags to `make` the output and library files
* Create a `.py` file which imports, implements the library and casts any necessary variables

### API out of line

* Create a build file in python. The `set_source` can be a block of C code or a `.h` header file
* Run the build file
* Import library

## <center> [Exercises](./04-Exercises-cffi.ipynb) </center>